# 01 importing libraries

In [35]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq 
import os

In [36]:
# importing the  data frame
customer_df = pd.read_csv('customers.csv')

In [37]:
# checking the columns of the dataframe
customer_df.head(15)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [38]:
# checking the df's dimension
customer_df.shape

(206209, 10)

In [39]:
# df's basic statistics
customer_df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [40]:
# df's basic informations
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


# 04-Wrangling the data: Checking the consistent logic;e.g.: renaming columns with illogical names and dropping columns that don’t add anything to my analysis.

In [41]:
# once more checking the head of the df
customer_df.head(2)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285


In [42]:
# revisig the column names since they need to be revised and unified
customer_df.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'surname','Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age',
                               'n_dependants' : 'number_of_dependants', 'fam_status' : 'marital_status'}, inplace = True)

In [43]:
# check the adjusted outcome
customer_df.head(2)

,user_id,first_name,surname,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285


In [45]:
# dropping unneccessary columns
# in this step, I decided to drop first_name & surname columns, due to several reasons: a) each customer has a unique user_id 
# which makes it trackable easliy. So, based on current demands, having the whole name of customers are unnecessary.
# b) since I have to merge this df with another on in the second part, the key column for this action is user_id again. Besides, 
# dropping the two columns make the final df a little bit shorter and smaller. 
# I'll do the dropping through an override to a new object, this way, the first df will remain intact.

cust_df_wrangled = customer_df.drop(columns=["first_name", "surname"])


In [46]:
# checking the result
cust_df_wrangled.head(2)

,user_id,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285


Here is the end of data wrangling.

# 05- checking the data quality and consistency: address missing values and duplicates, and convert any mixed-type data.

In [47]:
# checking missing values
cust_df_wrangled.isnull().sum()

user_id                 0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

In [48]:
# checking the duplicates
cust_df_wrangled.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
206204    False
206205    False
206206    False
206207    False
206208    False
Length: 206209, dtype: bool

In [49]:
# once more checking the duplicates to double-check
cust_df_wrangled.duplicated().sum()

np.int64(0)

In [50]:
#checking the mixed type data
cust_df_wrangled.dtypes

user_id                  int64
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

Here, by the end of data consistancy check, hopefully there was no missing, duplicated and mixed type values.

# 06- combining the customer dataframe with the rest of the Instacart data set already prepraed during this achievment.

 I have seen in other student's assignment that they used a lighter version of the previous dataframe, therefore, I searched 
about it and made it in my project too. The .parquet version is around 355 MB while the pickle version is 5 GB!

In [ ]:
# because of memory issues, I had to import the df in the following 2 steps:
# step 1
table = pq.read_table(
   "orders_products_merged_aggre_grouped_final.parquet",
      memory_map=True)

In [ ]:
# step2
df_ords_prods_merge = table.to_pandas(
    split_blocks=True,
    self_destruct=True
)

In [ ]:
# to ckeck deeper the memory usage and making more amendments
df_ords_prods_merge.info(memory_usage="deep")

In the following steps, I had to make some changes in order to make this dataframe much more lughter to avoid any further 
VS Code crashes.

In [24]:
object_cols = [
    "eval_set",
    "product_name",
    "price_range_loc",
    "busiest day",
    "busiest days",
    "busiest_period_of_day",
    "loyalty_flag",
    "spending_flag",
    "order_frequency_flag",
]

for col in object_cols:
    df_ords_prods_merge[col] = df_ords_prods_merge[col].astype("category")


In [25]:
int_cols = [
    "order_id",
    "user_id",
    "order_number",
    "orders_day_of_week",
    "order_hour_of_day",
    "product_id",
    "add_to_cart_order",
    "reordered",
    "aisle_id",
    "department_id",
    "max_order",
]

for col in int_cols:
    df_ords_prods_merge[col] = pd.to_numeric(df_ords_prods_merge[col], downcast="integer")


In [26]:
float_cols = [
    "days_since_prior_order",
    "prices",
    "avg_product_price",
    "median_days_of_orders",
]

for col in float_cols:
    df_ords_prods_merge[col] = pd.to_numeric(df_ords_prods_merge[col], downcast="float")


In [ ]:
# I need to recheck the results of the changes to see how much I could save on the memory 
df_ords_prods_merge.info(memory_usage="deep")


In [28]:
df_ords_prods_merge.to_parquet(
    "orders_products_merged_aggre_grouped_optimized.parquet",
    compression="snappy"
)


Now I can move on to the further steps.

In [29]:
# quick check on the imported data frame
df_ords_prods_merge.shape

(32404859, 26)

In [30]:
# continue...
df_ords_prods_merge.head(5)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_product_price,spending_flag,median_days_of_orders,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,prior,1,2,8,NaN,14084,2,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,prior,1,2,8,NaN,12427,3,0,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,prior,1,2,8,NaN,26088,4,0,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,prior,1,2,8,NaN,26405,5,0,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [31]:
df_ords_prods_merge.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', '_merge', 'product_name', 'aisle_id',
       'department_id', 'prices', 'merge_source', 'price_range_loc',
       'busiest day', 'busiest days', 'busiest_period_of_day', 'max_order',
       'loyalty_flag', 'avg_product_price', 'spending_flag',
       'median_days_of_orders', 'order_frequency_flag'],
      dtype='object')

In [32]:
# to make this df smaller in size, I would drop two columns that I consider useless here: _merge & merge_source.
df_ords_prods_merge_2 = df_ords_prods_merge.drop(columns=["_merge" , "merge_source"])


In [33]:
# checking the result of the dropping
df_ords_prods_merge_2.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest day',
       'busiest days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_product_price', 'spending_flag', 'median_days_of_orders',
       'order_frequency_flag'],
      dtype='object')

In [ ]:
# before merging the 2 dataframes; I have to be assure about the dtype of both key columns
df_ords_prods_merge_2["user_id"].dtype
cust_df_wrangled["user_id"].dtype


dtype('int64')

In [55]:
# now I can merge the 2 dataframes
df_combined = df_ords_prods_merge_2.merge(cust_df_wrangled, on="user_id", how="inner")

In [56]:
df_combined.shape

(32404859, 31)

In [57]:
df_combined.head(2)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,spending_flag,median_days_of_orders,order_frequency_flag,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,prior,1,2,8,NaN,14084,2,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [58]:
# exporting this merged dataframe to use in the second part of the assignment
df_combined.to_parquet(
    "orders_customers_combined.parquet",
    compression="snappy",
    engine="pyarrow"
)
